<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-15 16:07:29
-------------------
qualified stocks: 95
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  2.21 L
Current:  1.27 C
-------------------
Today PnL: 1.16 L (0.92%)
Current PnL: -16.92 L (-12.0%)
CY Booked + Current PnL: -14.05 L (-9.96%)
-------------------
Total profit:  4.19 L
Total loss:  -21.11 L
-------------------
Total Booked + Current PnL: 12.93 L (11.34%)
Total Booked PnL: 29.85 L (26.17%)
Curr Year Booked PnL: 2.87 L (2.26%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.06 C
Est FTT PnL: 79.38 L (62.65%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.41 C (111.21%)
Deployed:  1.14 C
Current:  1.27 C
CAGR/XIRR %: 7.6%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.12,14.94,X-MC,86.32,141140.0,26362.0,9358.0,0.05,22.97,6.63,31.13,220.0,2.82,1.13,37.48,AR,ATH,TYRES
62,RELIANCE,1288.53,1526.00,6.87,X-LC,96.84,229764.0,26176.0,11350.0,2.09,12.86,4.94,18.43,157.0,2.31,1.85,25.16,XY25,NTT,REFINERIES
69,SBILIFE,1496.49,1924.99,0.88,H-LC,81.05,149906.0,24201.0,11798.0,1.67,19.25,7.87,28.63,147.0,2.05,1.20,33.55,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-15.79,X-LC,37.89,93345.0,18223.0,13936.0,0.09,24.26,14.93,42.81,120.0,1.31,0.75,17.07,X40,BTT,CHEMICALS
70,SBIN,760.30,863.00,-12.51,M-LC,48.42,214054.0,12574.0,14641.0,0.88,6.24,6.84,13.51,60.0,0.86,1.72,17.27,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VIPIND,488.80,718.00,-848.59,H-SC,100.00,71331.0,-23985.0,68677.0,8.29,-25.16,96.28,46.89,82.0,-0.35,0.57,40.69,X40N,NTT,MISC
89,VALIANTORG,512.64,838.00,-310.23,H-SC,9.47,89375.0,-43911.0,128503.0,5.17,-32.95,143.78,63.47,161.0,-0.34,0.72,49.81,XR,NTT,CHEMICALS
34,HONAUT,42646.15,58518.52,-22.39,X-MC,74.74,113685.0,-14253.0,61867.0,5.03,-11.14,54.42,37.22,73.0,-0.23,0.91,16.07,X40N,ATH,ELECTRICAL
81,TATAMOTORS,776.02,1065.00,-52.03,M-LC,95.79,192946.0,-12699.0,89276.0,4.17,-6.18,46.27,37.24,1.0,-0.14,1.55,25.59,XY24,NTT,AUTO
76,STARHEALTH,551.56,761.00,-6.81,M-SC,67.37,152496.0,-47720.0,123751.0,4.09,-23.83,81.15,37.97,62.0,-0.39,1.22,22.44,XY24,NTT,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,QUESS,711.66,986.00,NaN,NaN,97.89,78260.0,-77594.0,137675.0,-2.96,-49.79,175.92,38.55,NaN,-0.56,0.63,19.84,XY24,NTT,MISC
26,EASEMYTRIP,18.12,26.40,20.84,M-SC,1.05,116315.0,-60500.0,141299.0,-2.38,-34.22,121.48,45.70,29.0,-0.43,0.93,7.97,XY24,NTT,TRAVEL
77,SURYODAY,189.78,240.00,42.54,H-SC,53.68,100780.0,-49146.0,88817.0,-2.25,-32.78,88.13,26.46,88.0,-0.55,0.81,29.11,XR,NTT,BANKS
87,UNIONBANK,123.87,163.00,-14.11,M-LC,58.95,150277.0,9437.0,35060.0,-2.05,6.70,23.33,31.59,48.0,0.27,1.21,30.54,XY24,NTT,BANKS
79,TANLA,917.30,1963.11,-40.61,H-SC,93.68,145233.0,-94182.0,367135.0,-1.99,-39.34,252.79,114.01,20.0,-0.26,1.17,33.20,AR,ATH,IT


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAMOTORS,776.02,1065.00,-52.03,M-LC,95.79,192946.0,-12699.0,89276.0,4.17,-6.18,46.27,37.24,1.0,-0.14,1.55,25.59,XY24,NTT,AUTO
37,INDIAMART,2311.97,4911.36,-52.59,H-SC,65.26,102542.0,815.0,113555.0,2.04,0.80,110.74,112.43,2.0,0.01,0.82,21.02,AR,ATH,MISC
2,ABB,5319.37,8953.12,-37.34,H-LC,70.53,143375.0,10391.0,80448.0,1.77,7.81,56.11,68.31,4.0,0.13,1.15,16.91,AR,ATH,ELECTRICAL
82,TCS,3928.29,4998.00,-14.23,X-LC,71.58,193315.0,-18813.0,76572.0,0.88,-8.87,39.61,27.23,5.0,-0.25,1.55,10.78,X40,BTT,IT
48,LICI,777.74,983.00,-15.35,H-LC,57.89,188227.0,14013.0,31961.0,2.20,8.04,16.98,26.39,6.0,0.44,1.51,15.24,XY25,NTT,INSURANCE


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.12,14.94,X-MC,86.32,141140.0,26362.0,9358.0,0.05,22.97,6.63,31.13,220.0,2.82,1.13,37.48,AR,ATH,TYRES
52,MEDANTA,1022.60,1486.00,7.25,X-MC,12.63,188982.0,33547.0,36889.0,0.03,21.58,19.52,45.32,201.0,0.91,1.52,28.59,XY24,NTT,HEALTHCARE
56,PIDILITIND,2504.06,3576.00,-15.79,X-LC,37.89,93345.0,18223.0,13936.0,0.09,24.26,14.93,42.81,120.0,1.31,0.75,17.07,X40,BTT,CHEMICALS
57,POLYCAB,5005.92,7541.35,8.17,H-MC,88.42,169176.0,29010.0,41989.0,1.02,20.70,24.82,50.65,55.0,0.69,1.36,29.11,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-0.40,M-LC,91.58,192649.0,41443.0,24216.0,2.51,27.41,12.57,43.43,37.0,1.71,1.55,33.54,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LUXIND,1448.19,2442.00,33.16,M-SC,25.26,138216.0,638.0,93780.0,-0.07,0.46,67.85,68.62,79.0,0.01,1.11,19.82,X40N,NTT,TEXTILES
57,POLYCAB,5005.92,7541.35,8.17,H-MC,88.42,169176.0,29010.0,41989.0,1.02,20.70,24.82,50.65,55.0,0.69,1.36,29.11,X40N,ATH,CABLES
6,ANGELONE,2328.67,3033.00,2.48,H-SC,72.63,166959.0,15595.0,30186.0,0.78,10.30,18.08,30.25,111.0,0.52,1.34,32.14,X40N,NTT,FINANCE
4,ADANIPORTS,1103.69,1583.00,-0.40,M-LC,91.58,192649.0,41443.0,24216.0,2.51,27.41,12.57,43.43,37.0,1.71,1.55,33.54,XY24,NTT,MISC
28,GILLETTE,8109.44,10330.69,-4.76,H-SC,35.79,140530.0,2670.0,35090.0,0.65,1.94,24.97,27.39,68.0,0.08,1.13,24.68,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
94,WIPRO,243.46,420.00,-6.99,M-LC,43.16,159073.0,8128.0,101330.0,1.43,5.38,63.70,72.51,52.0,0.08,1.28,19.77,XR,NTT,IT
18,CAMS,3643.00,5250.99,-7.11,H-SC,11.58,104339.0,2335.0,42685.0,2.12,2.29,40.91,44.14,22.0,0.05,0.84,18.90,XR,ATH,MISC
37,INDIAMART,2311.97,4911.36,-52.59,H-SC,65.26,102542.0,815.0,113555.0,2.04,0.80,110.74,112.43,2.0,0.01,0.82,21.02,AR,ATH,MISC
12,BANKINDIA,113.49,190.00,-34.31,M-MC,32.63,129789.0,637.0,86426.0,0.17,0.49,66.59,67.42,41.0,0.01,1.04,26.23,XR,NTT,BANKS
92,VOLTAS,1272.00,1929.20,-30.60,H-MC,14.74,138864.0,-1056.0,73348.0,1.11,-0.75,52.82,51.67,32.0,-0.01,1.12,4.89,AR,ATH,AC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,UJJIVANSFB,52.77,60.00,41.19,M-SC,82.11,122607.0,-19872.0,39394.0,1.14,-13.95,32.13,13.70,214.0,-0.50,0.98,44.71,X40N,NTT,BANKS
53,MRF,114777.60,150504.12,14.94,X-MC,86.32,141140.0,26362.0,9358.0,0.05,22.97,6.63,31.13,220.0,2.82,1.13,37.48,AR,ATH,TYRES
69,SBILIFE,1496.49,1924.99,0.88,H-LC,81.05,149906.0,24201.0,11798.0,1.67,19.25,7.87,28.63,147.0,2.05,1.20,33.55,AR,ATH,INSURANCE
87,UNIONBANK,123.87,163.00,-14.11,M-LC,58.95,150277.0,9437.0,35060.0,-2.05,6.70,23.33,31.59,48.0,0.27,1.21,30.54,XY24,NTT,BANKS
14,BERGEPAINT,558.54,680.00,-15.50,H-MC,62.11,181403.0,4346.0,34158.0,3.49,2.45,18.83,21.75,154.0,0.13,1.46,29.76,XY24,NTT,PAINTS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VIPIND,488.80,718.0,-848.59,H-SC,100.00,71331.0,-23985.0,68677.0,8.29,-25.16,96.28,46.89,82.0,-0.35,0.57,40.69,X40N,NTT,MISC
80,TATAELXSI,7332.28,9000.0,-17.66,X-MC,98.95,86527.0,-16125.0,39474.0,1.31,-15.71,45.62,22.74,47.0,-0.41,0.69,30.06,X40N,NTT,IT
58,QUESS,711.66,986.0,NaN,NaN,97.89,78260.0,-77594.0,137675.0,-2.96,-49.79,175.92,38.55,NaN,-0.56,0.63,19.84,XY24,NTT,MISC
62,RELIANCE,1288.53,1526.0,6.87,X-LC,96.84,229764.0,26176.0,11350.0,2.09,12.86,4.94,18.43,157.0,2.31,1.85,25.16,XY25,NTT,REFINERIES
81,TATAMOTORS,776.02,1065.0,-52.03,M-LC,95.79,192946.0,-12699.0,89276.0,4.17,-6.18,46.27,37.24,1.0,-0.14,1.55,25.59,XY24,NTT,AUTO


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.51
1,25,39.25
2,50,68.34


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.98
X40N    15.30
X40     14.49
AR      14.07
XR      10.37
XY25     8.42
X200     1.21
SR       1.11
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.47
M-SC    17.81
X-LC    16.40
M-LC     8.48
H-MC     7.88
X-MC     6.79
H-LC     5.95
M-MC     5.57
X-SC     2.82
L-SC     2.28
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.73,-1.63,33.95
BANKS,11.10,-8.95,62.39
IT,9.31,-19.29,85.85
FINANCE,7.47,-18.85,72.48
MISC,7.21,-26.27,80.60
INSURANCE,6.15,-0.56,30.87
PAINTS,5.66,-16.85,51.43
ELECTRICAL,4.14,-7.55,62.36
HEALTHCARE,3.64,-2.66,35.81


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2994301.0,27
XR,1255009.0,15
X40N,1254717.0,18
AR,1251290.0,15
X40,787970.0,12
SR,206752.0,2
XY25,121326.0,5
X200,66855.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2389725.0,22
M-SC,2017227.0,21
X-LC,698960.0,12
M-MC,573429.0,5
H-MC,415557.0,7
X-SC,357977.0,4
M-LC,325032.0,6
L-SC,314557.0,4
X-MC,308970.0,6


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      913146.0      6
M-SC       XY24      760490.0      6
H-SC       X40N      543503.0      7
           AR        541564.0      3
M-SC       X40N      450074.0      6
           XR        428290.0      5
M-MC       XY24      404357.0      3
X-LC       X40       369215.0      5
H-SC       XR        298792.0      4
M-LC       XY24      209061.0      4
M-SC       X40       198881.0      2
H-LC       AR        197967.0      4
X-LC       AR        188166.0      3
H-MC       XY24      172591.0      2
L-SC       XR        162914.0      2
X-MC       XY24      140636.0      2
X-SC       SR        121020.0      1
           XR        119147.0      1
           X40N      117810.0      2
X-MC       X40N      101341.0      2
M-LC       XR        101330.0      1
M-SC       AR         93760.0      1
H-SC       X40        92720.0      2
L-SC       XY24       87162.0      1
M-MC       XR         86426.0      1
M-SC       SR         85732.0      1
M-MC       AR         82646.0      1
X-LC       XY25       74724.0      3
H-MC       AR         73348.0      1
           X40        69519.0      2
X-LC       X200       66855.0      1
L-SC       AR         64481.0      1
H-MC       XR         58110.0      1
X-MC       X40        57635.0      1
H-MC       X40N       41989.0      1
H-LC       XY25       31961.0      1
M-LC       XY25       14641.0      1
X-MC       AR          9358.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
